In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *

Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/L1norm.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/Train.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/random.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/decay_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/two_to_four.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Thinet.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Global_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Ln_structured.py


In [2]:
dataset = MNIST()
train_loader, test_loader = dataset.get_dataloader()

In [3]:
#train model and return it
model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = Trainer(model, 5, train_loader, criterion, optimizer)
original_model = model.train()
#print("Accuracy of original model : ", accuracy(original_model, test_loader))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 4.9743 | 
Epoch: 2 | train_loss: 3.6368 | 
Epoch: 3 | train_loss: 3.2351 | 
Epoch: 4 | train_loss: 2.9756 | 
Epoch: 5 | train_loss: 2.7218 | 


In [4]:
#for UnstructuredL1normPrune

# model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(original_model.parameters(), lr=0.001)

trainer = UnstructuredL1normPrune(original_model, 5, train_loader, criterion, optimizer, 0.9)

#train and prune the model and return both the original and pruned model
#original_model, pruned_model = trainer.train_prune_retrain()
pruned_model = trainer.prune_model()

In [5]:
%%bash --bg 
while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt; sleep 0.1; done

In [6]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))

with open('unpruned_output.txt', 'a') as f:
    f.write("\energy computation for original model completed.\n")

Accuracy of original model :  [98.73]


In [7]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt'" | xargs -r kill

In [8]:
#for pruned 

In [9]:
%%bash --bg 
while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt; sleep 0.1; done

In [10]:
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))

with open('pruned_output.txt', 'a') as f:
    f.write("\energy computation for pruned model completed.\n")

Accuracy of pruned model :  [27.24]


In [11]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt'" | xargs -r kill

In [ ]:
#done